In [85]:
import requests
import pandas as pd

In [86]:
def query(request):
    request['action'] = 'query'
    request['format'] = 'json'
    lastContinue = {}
    while True:
        # Clone original request
        req = request.copy()
        # Modify it with the values returned in the 'continue' section of the last result.
        req.update(lastContinue)
        # Call API
        result = requests.get('https://grasswiki.osgeo.org/w/api.php', params=req).json()
        if 'error' in result:
            raise Error(result['error'])
        if 'warnings' in result:
            print(result['warnings'])
        if 'query' in result:
            yield result['query']
        if 'continue' not in result:
            break
        lastContinue = result['continue']

In [76]:
for result in query({'generator': 'allpages', "prop": "info|images"}):
#     for each in result["pages"]:
    print(result["pages"].keys())
    print("\n")

dict_keys(['2999', '3354', '4515', '3652', '4477', '2079', '1998', '1543', '1414', '2923'])


dict_keys(['3152', '3173', '3212', '3213', '3156', '3153', '3155', '2922', '3215', '3161'])


dict_keys(['3660', '3214', '3174', '3181', '3180', '3182', '3178', '3179', '1614', '1388'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2855', '2000', '2468', '2154', '4312', '4385', '3489', '1837', '1714', '1789'])


dict_keys(['2078', '2312', '1783', '1790', '1477', '1836', '

KeyboardInterrupt: 

In [87]:
data = []
pageids = []
for result in query({'generator': 'allpages', "prop": "info|images|revisions"}):
    for each in result["pages"]:
        item = result["pages"][each]
        if (item['pageid'] in pageids):
            continue
        pageids.append(item['pageid'])
        timestamp = item['revisions'][0]['timestamp'].split('T')[0] if 'revisions' in item else None
        length = item.get("length", None)
        record = (item['pageid'], item['title'], item.get('pagelanguage', None), timestamp,
              not bool(item.get('redirect', 1)), length, len(item.get('images', [])))

        data.append(record)
df = pd.DataFrame.from_records(data, index='pageid', columns=['pageid', 'title', 'language', 'last_revision', 'redirects', 'length', 'image_number'])

In [78]:
df.loc[2855]

title            Add point to vector map at given position
language                                                en
last_revision                                   2013-12-31
redirects                                            False
length                                                 932
image_number                                             0
Name: 2855, dtype: object

In [88]:
df2 = df.copy()
df2['wikilinks'] = None
for result in query({'generator': 'allpages', "prop": "links"}):
    for each in result["pages"]:
        item = result["pages"][each]
        df2.at[item['pageid'], 'wikilinks'] = len(item.get('links', []))
        


In [89]:
df3 = df2.copy()
df3['categories'] = None
for result in query({'generator': 'allpages', "prop": "categories"}):
    for each in result["pages"]:
        item = result["pages"][each]
        if 'categories' in item:
            cats = []
            for cat in item['categories']:
                cats.append(cat['title'].split(':', 1)[1])
            df3.at[item['pageid'], 'categories'] = ','.join(cats)
            

In [90]:
df4 = df3.copy()
df4['num_revisions'] = None
df4['created'] = None
for index, row in df.iterrows():
    for result in query({"titles": row['title'], "prop": "revisions", "rvprop": "timestamp", "rvdir" : "newer", "rvlimit": 100}):
        try:
            revisions = result['pages'][str(index)]['revisions']
        except KeyError:
            df4.at[index, 'num_revisions'] = 0
            df4.at[index, 'created'] = 'unknown'
        df4.at[index, 'num_revisions'] = len(revisions)
        df4.at[index, 'created'] = revisions[0]['timestamp'].split('T')[0]
        

In [82]:
df4

,title,language,last_revision,redirects,length,image_number,wikilinks,categories,num_revisions,created
pageid,,,,,,,,,,
2999,/GRASS SoC Ideas,en,2012-04-13,True,34,0,0,None,1,2012-04-13
3354,3D,en,2013-04-13,True,26,0,0,None,1,2013-04-13
4515,ADCIRC,en,2021-01-06,False,9109,0,1,"Geodata,Import,Pages using deprecated source tags",38,2019-07-17
3652,AGU Fall Meeting 2014: GRASS related presentat...,en,2016-02-29,False,1790,0,0,"2014,Conferences,Workshops",6,2014-12-18
4477,AGU Fall Meeting 2018,en,2019-03-03,False,17261,9,0,"2018,Conferences",16,2018-12-23
...,...,...,...,...,...,...,...,...,...,...
3698,XVI Meeting degli Utenti Italiani di GRASS GFO...,en,2015-02-18,False,1291,0,2,Workshops,6,2015-02-02
3697,XVI Meeting degli Utenti Italiani di GRASS GFO...,en,2015-02-26,False,2716,3,4,None,22,2015-02-02
3696,XVI Meeting degli Utenti Italiani di GRASS GFO...,en,2015-02-18,False,2290,0,0,None,18,2015-02-02


In [91]:
df4.to_csv("grasswiki_pages2.csv")

In [84]:
!head grasswiki_pages.csv

pageid,title,language,last_revision,redirects,length,image_number,wikilinks,categories,num_revisions,created
2999,/GRASS SoC Ideas,en,2012-04-13,True,34,0,0,,1,2012-04-13
3354,3D,en,2013-04-13,True,26,0,0,,1,2013-04-13
4515,ADCIRC,en,2021-01-06,False,9109,0,1,"Geodata,Import,Pages using deprecated source tags",38,2019-07-17
3652,AGU Fall Meeting 2014: GRASS related presentations and posters,en,2016-02-29,False,1790,0,0,"2014,Conferences,Workshops",6,2014-12-18
4477,AGU Fall Meeting 2018,en,2019-03-03,False,17261,9,0,"2018,Conferences",16,2018-12-23
2079,ASTER topography,en,2013-02-19,False,3451,0,0,Tutorial,27,2009-07-04
1998,AVHRR,en,2012-12-20,False,7072,0,0,"Geodata,Image processing,Import",9,2009-03-13
1543,About GRASS,en,2009-08-25,False,598,0,0,,47,2006-12-31
1414,AddOns,en,2020-06-25,False,6360,0,0,"Community,Development,Documentation,Installation,Pages using deprecated source tags",74,2012-04-09
